In [1]:
import pandas_datareader as pdr
from datetime import datetime



In [2]:
import tensorflow as tf
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
import autoencoder
import model
from datetime import datetime
from datetime import timedelta
sns.set()

In [3]:
companies = ['INTC', 'AMD', 'CSCO', 'AAPL', 'MU', 'NVDA', 'QCOM', 'AMZN', 'NFLX', 'FB', 'GOOG', 'BABA', 'EBAY', 'IBM', 'XLNX', 'TXN', 'NOK', 'TSLA', 'MSFT', 'SNPS']
len(companies)

20

In [24]:
comp_data = {}
for ticker in companies:
    tmp = pdr.get_data_yahoo(symbols=ticker, start=datetime(2018, 2, 1), end=datetime(2019, 5, 9))
    comp_data[ticker] = tmp
    #print(len(tmp))

oil = pdr.get_data_yahoo(symbols='OIL', start=datetime(2018, 2, 1), end=datetime(2019, 5, 9))
len(oil)
    

320

import numpy as np

class Model:
    def __init__(self, learning_rate, num_layers, size, size_layer, output_size, forget_bias = 0.1):
        
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(size_layer) for _ in range(num_layers)], state_is_tuple = False)
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop = tf.contrib.rnn.DropoutWrapper(rnn_cells, output_keep_prob = forget_bias)
        self.hidden_layer = tf.placeholder(tf.float32, (None, num_layers * 2 * size_layer))
        self.outputs, self.last_state = tf.nn.dynamic_rnn(drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32)
        rnn_W = tf.Variable(tf.random_normal((size_layer, output_size)))
        rnn_B = tf.Variable(tf.random_normal([output_size]))
        self.logits = tf.matmul(self.outputs[-1], rnn_W) + rnn_B
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)

num_layers = 1
size_layer = 128
timestamp = 1
epoch = 500
dropout_rate = 0.1

Import ARIMA model using stats model



import statsmodels.api as sm
from itertools import product
from scipy import stats

def reverse_close(array):
    return minmax.inverse_transform(array.reshape((-1,1))).reshape((-1))

Rise: CP
𝑡𝑜𝑑𝑎𝑦 > CP𝑦𝑒𝑠𝑡𝑒𝑟𝑑𝑎𝑦 , 𝑅𝑐ℎ𝑎𝑛𝑔𝑒 > 2%
◦ Remain Stable: 𝑅
𝑐ℎ𝑎𝑛𝑔𝑒 ≤ 2%
◦ Fall: CP
𝑡𝑜𝑑𝑎𝑦 < CP𝑦𝑒𝑠𝑡𝑒𝑟𝑑𝑎𝑦 , 𝑅𝑐ℎ𝑎𝑛𝑔𝑒 > 2%

In [9]:
def pred_stock(pred_close_price, today_close_price):
    
    percent_change = abs(pred_close_price - today_close_price) * 100/ today_close_price
    
    if percent_change > 2.0:
        if (pred_close_price > today_close_price):
            return 0 # rise
        elif (pred_close_price <= today_close_price):
            return 2 # fall
    elif percent_change < 2.0:
        return 1 # stable



intl = comp_data['INTC']
intl['oil_high'] = oil['High']
intl['oil_low'] = oil['Low']
intl['oil_open'] = oil['Open']
intl['oil_close'] = oil['Close']

final_res = []
for cpn in companies:
    
    intl = comp_data[cpn]
    intl['oil_high'] = oil['High']
    intl['oil_low'] = oil['Low']
    intl['oil_open'] = oil['Open']
    intl['oil_close'] = oil['Close']
    
    y_train = intl.Close[1:]
    x_train = intl.iloc[:-1,:]
    x_test = intl.iloc[-1, :]
    
    df_x_test = MinMaxScaler().fit_transform(x_test.astype('float32'))
    df_x_test = pd.DataFrame(df_x_test)
    thought_vector_x_test = autoencoder.reducedimension(df_x_test.values, 3, 0.001, 128, 100)
    
    minmax = MinMaxScaler().fit(y_train.values.reshape((-1,1)))
    df_log = MinMaxScaler().fit_transform(x_train.astype('float32'))
    df_log = pd.DataFrame(df_log)
    
    #minmax = MinMaxScaler().fit(intl.iloc[:, 3].values.reshape((-1,1)))
    
    ##df_log = MinMaxScaler().fit_transform(intl.iloc[:, 1:].astype('float32'))
    
    #df_log = MinMaxScaler().fit_transform(intl.astype('float32'))

    #df_log = pd.DataFrame(df_log)

    thought_vector = autoencoder.reducedimension(df_log.values, 3, 0.001, 128, 100)

    tf.reset_default_graph()

    modelnn = Model(0.01, num_layers, thought_vector.shape[1], size_layer, 1, dropout_rate)
    
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch):
        init_value = np.zeros((1, num_layers * 2 * size_layer))
        total_loss = 0
        for k in range(0, (thought_vector.shape[0] // timestamp) * timestamp, timestamp):
            
            batch_x = np.expand_dims(thought_vector[k: k + timestamp, :], axis = 0)
            
            #batch_y = df_log.values[k + 1: k + timestamp + 1, 3].reshape([-1, 1])
            batch_y = y_train.values[k : k + timestamp].reshape([-1, 1])

            last_state, _, loss = sess.run([modelnn.last_state, 
                                            modelnn.optimizer, 
                                            modelnn.cost], feed_dict={modelnn.X: batch_x, 
                                                                      modelnn.Y: batch_y, 
                                                                      modelnn.hidden_layer: init_value})
            init_value = last_state
            total_loss += loss
        total_loss /= (thought_vector.shape[0] // timestamp)
        #if (i + 1) % 100 == 0:
         #   print('epoch:', i + 1, 'avg loss:', total_loss)

    #output_predict = np.zeros(((thought_vector.shape[0] // timestamp) * timestamp, 1))
    output_predict = np.zeros(((thought_vector_x_test.shape[0] // timestamp) * timestamp, 1))

    init_value = np.zeros((1, num_layers * 2 * size_layer))
    tmp1 = (thought_vector.shape[0] // timestamp) * timestamp
    #tmp2 = len(df_log)
    
    tmp2 = len(df_x_test)
    tmp3_go = tmp2 - tmp1
    
    for k in range(0, (thought_vector.shape[0] // timestamp) * timestamp, timestamp):
        out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], 
            feed_dict = {modelnn.X:np.expand_dims(thought_vector_x_test[k + tmp3_go: k + tmp3_go + timestamp, :], axis = 0),
                                         modelnn.hidden_layer: init_value})
        init_value = last_state
        output_predict[k: k + timestamp, :] = out_logits
        
    #sess.close()
    
    Qs = range(0, 1)
    qs = range(0, 2)
    Ps = range(0, 2)
    ps = range(0, 2)
    D=1
    parameters = product(ps, qs, Ps, Qs)
    parameters_list = list(parameters)
    best_aic = float("inf")
    for param in parameters_list:
        try:
            #arima=sm.tsa.statespace.SARIMAX(df_log.iloc[:,3].values, order=(param[0], D, param[1]), seasonal_order=(param[2], D, param[3], 1)).fit(disp=-1)
            arima=sm.tsa.statespace.SARIMAX(y_train.values, order=(param[0], D, param[1]), seasonal_order=(param[2], D, param[3], 1)).fit(disp=-1)

        except:
            continue
        aic = arima.aic
        if aic < best_aic and aic:
            best_arima = arima
            best_aic = aic

    #best_aic


    pred_arima = best_arima.predict()



    boundary = (thought_vector.shape[0] // timestamp) * timestamp
    start = thought_vector.shape[0] - boundary
    #stack_predict = np.vstack([pred_arima[:boundary], output_predict.reshape((-1))]).T
    stack_predict = np.vstack([pred_arima[-1], output_predict.reshape((-1))]).T

    #where_below_0 = np.where(stack_predict < 0)
    #where_higher_1 = np.where(stack_predict > 1)
    #stack_predict[where_below_0[0], where_below_0[1]] = 0
    #stack_predict[where_higher_1[0], where_higher_1[1]] = 1

    params_xgd = {
        'max_depth': 7,
        'objective': 'reg:logistic',
        'learning_rate': 0.05,
        'n_estimators': 10000
        }
    
    train_Y = df_log.values[start:, 3]

    clf = xgb.XGBRegressor(**params_xgd) ## **?

    clf.fit(stack_predict,train_Y, eval_set=[(stack_predict,train_Y)], 
            eval_metric='rmse', early_stopping_rounds=20, verbose=False)
    
    stacked = clf.predict(stack_predict)

    true_close = reverse_close(train_Y)
    pred_close = reverse_close(stacked) # 
    #pred_ = pred_arima[-1]
    
    pred_ = pred_close[-1] # t + 1 price
    #pred_ = pred_close
    today_ = true_close[-1] # t price
    wow_pred = pred_stock(pred_, today_)
    final_res.append(wow_pred)
    print('====> {}: {} ----> today: {} --- pred_tomorrow: {}'.format(cpn, wow_pred, today_, pred_))

In [31]:
final_res = []
for cpn in companies:
    
    intl = comp_data[cpn]
    intl['oil_high'] = oil['High']
    intl['oil_low'] = oil['Low']
    intl['oil_open'] = oil['Open']
    intl['oil_close'] = oil['Close']
    
    y_train = intl.Close[1:]
    x_train = intl.iloc[:-1,:]
    #print("COMPANY\n", intl.tail())
    #print("y-TRAI\n", y_train.tail())
    #print("X-TRAIN\n", x_train.tail())
    y_train = y_train.values
    x_train = x_train.values
    
    x_test = intl.iloc[-1, :]
    #print("X-test\n", x_test)
    x_test = x_test.values.reshape(1, -1)
    #print(x_test)
    #df_x_test = MinMaxScaler().fit_transform(x_test.astype('float32'))
    #df_x_test = pd.DataFrame(df_x_test)
    
    #x_train = autoencoder.reducedimension(x_train, 5, 0.001, 128, 100)
    #x_test = autoencoder.reducedimension(x_test, 5, 0.001, 128, 100)
    
    #minmax = MinMaxScaler().fit(y_train.values.reshape((-1,1))) # y train
    
    #minmax = pd.DataFrame(minmax)
    #minmax = np.array(minmax)
    #minmax = pd.DataFrame(minmax.reshape((-1, 1)))
    
    #df_log = MinMaxScaler().fit_transform(x_train.astype('float32'))
    #df_log = pd.DataFrame(df_log) # x train
    
    #df_log = df_log.values
    
    #print('y_train minmax: {}'.format(len(minmax)))
    #print('df_log: {}'.format(len(df_log)))

    
    
    # xgbOost 
    params_xgd = {
        'max_depth': 7,
        'objective': 'reg:logistic',
        'learning_rate': 0.05,
        'n_estimators': 10000
        }
    
    #train_Y = df_log.values[start:, 3]

    #clf = xgb.XGBRegressor() ## **?
    
    #clf.fit(stack_predict,y_train)
    clf = xgb.XGBRegressor()
    #model = xgb.XGBRegressor()
    #model.fit(df_log,minmax)
    #print (model)

    clf.fit(x_train, y_train)
    
    stacked = clf.predict(x_test)

    #true_close = reverse_close(train_Y)
    #pred_close = reverse_close(stacked) # 
    
    #pred_ = pred_arima[-1]
    
    #pred_ = pred_close[-1] # t + 1 price
    
    pred_ = stacked
    today_ = y_train[-1] # t price
    
    wow_pred = pred_stock(pred_, today_)
    final_res.append(wow_pred)
    print('====> {}: {} ----> today: {} --- pred_tomorrow: {}'.format(cpn, wow_pred, today_, pred_))

====> INTC: 1 ----> today: 46.619998931884766 --- pred_tomorrow: [46.838593]
====> AMD: 1 ----> today: 27.209999084472656 --- pred_tomorrow: [26.95659]
====> CSCO: 1 ----> today: 52.91999816894531 --- pred_tomorrow: [53.004677]
====> AAPL: 1 ----> today: 200.72000122070312 --- pred_tomorrow: [202.88957]
====> MU: 1 ----> today: 39.27000045776367 --- pred_tomorrow: [38.75364]
====> NVDA: 2 ----> today: 170.19000244140625 --- pred_tomorrow: [165.81253]
====> QCOM: 1 ----> today: 83.77999877929688 --- pred_tomorrow: [84.59456]
====> AMZN: 1 ----> today: 1899.8699951171875 --- pred_tomorrow: [1905.7084]
====> NFLX: 1 ----> today: 362.75 --- pred_tomorrow: [360.71155]
====> FB: 1 ----> today: 188.64999389648438 --- pred_tomorrow: [188.98941]
====> GOOG: 1 ----> today: 1162.3800048828125 --- pred_tomorrow: [1163.8878]
====> BABA: 1 ----> today: 179.0399932861328 --- pred_tomorrow: [178.98433]
====> EBAY: 1 ----> today: 37.619998931884766 --- pred_tomorrow: [37.550915]
====> IBM: 1 ----> toda

In [28]:
final_res

[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

autoencoder not work
short term data not good
rnn not work

In [32]:
# download up-to-date stock prices for validate predcition above ( run after 9pm TW time)
comp_data_check_pred = {}
for ticker in companies:
    tmp = pdr.get_data_yahoo(symbols=ticker, start=datetime(2019, 5, 1), end=datetime(2019, 5, 10))
    comp_data_check_pred[ticker] = tmp
    #print(len(tmp))

In [33]:
# validate results of prediction everday
val_res = []
for cpn in comp_data_check_pred:
    intl = comp_data_check_pred[cpn]
    print(intl.tail())
    tmr = intl.iloc[-1, 3]
    yes = intl.iloc[-2, 3]
    pred_ = pred_stock(tmr, yes)
    val_res.append(pred_)
    #print('yes: {}'.format(yes))
    #print('tmr: {}'.format(tmr))
    print('\n{}: {}'.format(cpn, pred_))

                 High        Low       Open      Close      Volume  Adj Close
Date                                                                         
2019-05-06  51.279999  50.189999  50.250000  51.220001  26245200.0  51.220001
2019-05-07  50.880001  49.930000  50.599998  50.480000  29713400.0  50.480000
2019-05-08  50.790001  49.070000  50.200001  49.240002  36812400.0  49.240002
2019-05-09  48.290001  46.049999  47.900002  46.619999  59642200.0  46.619999
2019-05-10  46.799999  45.099998  46.439999  46.200001  42385800.0  46.200001

INTC: 1
                 High        Low       Open      Close    Volume  Adj Close
Date                                                                       
2019-05-06  27.500000  26.450001  26.719999  27.420000  70344100  27.420000
2019-05-07  27.350000  26.209999  27.200001  26.660000  75868800  26.660000
2019-05-08  27.709999  26.270000  26.410000  27.090000  65967500  27.090000
2019-05-09  27.379999  26.030001  26.700001  27.209999  73150900 

In [34]:
val_res

[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [35]:
final_res

[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1] # GT 5/6 7/20

[1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1] # GT 5/7 7/20

[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] # GT 5/8 17/20

[2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] # GT 5/9 18/20

[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] #GT 5/19 17/20

In [36]:
file_name = '0510_0760816'

with open(file_name, 'r') as f:
    x = f.read().splitlines()

In [37]:
pred_tmr = [int(tm) for tm in x]
print(pred_tmr)

[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [38]:
n_correct = 0
for i in range(len(val_res)):
    val = val_res[i]
    pre = pred_tmr[i]
    if val == pre:
        n_correct += 1
print('n_correct_prediction: {}'.format(n_correct))

n_correct_prediction: 17


In [29]:
file_name = '0510_0760816'
def output_txt(file_name, list_pred):
    
    with open(file_name, "w") as f:
        
        for pred in list_pred:

            f.write("%s\n"%(str(int(pred))))

    f.close()

In [30]:
output_txt(file_name, final_res)